# Big Idea 3 (Data Structures including List, Dictionaries, 2D arrays and Iteration)

## Leaderboard Database (Kaiden and Navan)
> How do you set up a database model?
### What is a database model
- A database model shows the ________ ________ of a database
- It fundamentally determines in which manner data can be __________, __________, and _________
- Some popular database models are relational models, object oriented models, hierarchial models, flat file models, and network models
- The one below is an Object-Relational Model which is a combination of a _________ model and a __________ ________ model

### Creating SQLAlchemy App

In [10]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# Setup of key Flask object (app)
app = Flask(__name__)

# Setup SQLAlchemy object and properties for the database (db)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///sqlite.db'
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
app.config["SECRET_KEY"] = "SECRET_KEY"
db = SQLAlchemy(app)

# Images storage
app.config["MAX_CONTENT_LENGTH"] = 5 * 1024 * 1024  # maximum size of uploaded content
app.config["UPLOAD_EXTENSIONS"] = [".jpg", ".png", ".gif"]  # supported file types
app.config["UPLOAD_FOLDER"] = "volumes/uploads/"  # location of user uploaded content


### Creating Database Template

In [11]:

from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.exc import IntegrityError
import json
from werkzeug.security import generate_password_hash, check_password_hash


# 
# Leaderboard DB class that maps leaderboard SQL table 
#
class Leaderboard(db.Model):
    __tablename__ = "leaderboard"

    # 
    # Leaderboard DB columns for easy, medium and hard points with user info
    #    
    id = Column(Integer, primary_key=True)
    _username = Column(String(255), unique=True, nullable=False)
    _password = Column(String(255), nullable=False)
    _pointsEasy = Column(Integer, nullable=False)
    _pointsMedium = Column(Integer, nullable=False)
    _pointsHard = Column(Integer, nullable=False)

    # 
    # Leaderboard DB class constructor 
    #
    def __init__(self, username, password, pointsEasy, pointsMedium, pointsHard):
        self._username = username
        self.set_password(password)
        self._pointsEasy = pointsEasy
        self._pointsMedium = pointsMedium
        self._pointsHard = pointsHard

    # 
    # Leaderboard DB class string representation of an object
    #
    def __repr__(self):
        return "<Leaderboard(id='%s', username='%s', pointsEasy='%s', pointsMedium='%s', pointsHard='%s')>" % (
            self.id,
            self.username,
            self.pointsEasy,
            self.pointsMedium,
            self.pointsHard,
        )

    # 
    # Returns Leaderboard username
    #    
    @property
    def username(self):
        return self._username

    # 
    # Sets Leaderboard username
    #        
    @username.setter
    def username(self, value):
        self._username = value

    # 
    # checks Leaderboard username valid
    #            
    def is_username(self, username):
        return self._username == username

    # 
    # Returns Leaderboard easy points
    #        
    @property
    def pointsEasy(self):
        return self._pointsEasy

    # 
    # Sets Leaderboard easy points
    #        
    @pointsEasy.setter
    def pointsEasy(self, value):
        self._pointsEasy = value

    # 
    # Sets Leaderboard medium points
    #            
    @property
    def pointsMedium(self):
        return self._pointsMedium

    # 
    # Sets Leaderboard medium points
    #        
    @pointsMedium.setter
    def pointsMedium(self, value):
        self._pointsMedium = value

    # 
    # Returns Leaderboard hard points
    #            
    @property
    def pointsHard(self):
        return self._pointsHard

    # 
    # Sets Leaderboard hard points
    #        
    @pointsHard.setter
    def pointsHard(self, value):
        self._pointsHard = value

    # 
    # Returns Leaderboard password
    #            
    @property
    def password(self):
        return self._password[0:10] + "..."

    # 
    # Sets Leaderboard password
    #        
    def set_password(self, password):
        self._password = generate_password_hash(password, method='sha512')

    # 
    # Checks Leaderboard password validity
    #            
    def is_password(self, password):
        result = check_password_hash(self._password, password)
        if result:
            return True
        else:
            return False

    # 
    # Converts Leaderboard to dictionary
    #            
    def to_dict(self):
        return {"id": self.id, "username": self.username, "password": self.password, "pointsEasy": self._pointsEasy, "pointsMedium": self._pointsMedium, "pointsHard": self._pointsHard}

    # 
    # Converts Leaderboard to string values
    #                
    def __str__(self):
        return json.dumps(self.read())

    # 
    # Creates Leaderboard database
    #                
    def create(self):
        try:
            db.session.add(self)
            db.session.commit()
            return self
        except IntegrityError:
            db.session.remove()
            return None
    # 
    # Returns Leaderboard name value pairs
    #            
    def read(self):
        return {
            "id": self.id,
            "username": self.name,
            "password": self.uid,
            "pointsEasy": self.pointsEasy,
            "pointsMedium": self.pointsMedium,
            "pointsHard": self.pointsHard
        }

    # 
    # Updates Leaderboard DB rows for points and user data
    #                
    def update(self, username="", password="", pointsEasy="", pointsMedium="", pointsHard=""):
        """only updates values with length"""
        if len(username) > 0:
            self.username = username
        if len(pointsEasy) > 0:
            self.pointsEasy = pointsEasy
        if len(pointsMedium) > 0:
            self.pointsMedium = pointsMedium
        if len(pointsHard) > 0:
            self.pointsHard = pointsHard
        if len(password) > 0:
            self.set_password(password)
        db.session.add(self)
        db.session.commit()
        return self

    # 
    # Delets Leaderboard row from teh DB
    #                
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

### Initialization of Data

In [12]:
#
# Initializes Leaderboard DB with test data
#   
def init_leaderboards():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        l1 = Leaderboard(username="bob", password="apple", pointsEasy=2, pointsMedium=5, pointsHard=3)
        l2 = Leaderboard(username="bobby", password="appley", pointsEasy=20, pointsMedium=50, pointsHard=30)
        l3 = Leaderboard(username="bobbert", password="appled", pointsEasy=200, pointsMedium=500, pointsHard=300)
        l4 = Leaderboard(username="bobruth", password="appler", pointsEasy=100, pointsMedium=300, pointsHard=500)
        leaderboards = [l1, l2, l3, l4]

        """Builds sample user/note(s) data"""
        for l in leaderboards:
            try:
                '''add user to table'''
                object = l.create()
                print(f"Created new uid {object.username}")
                db.session.add(l)
                db.session.commit()
            except:
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {l.username}, or error.")

init_leaderboards()

Created new uid bob
Created new uid bobby
Created new uid bobbert
Created new uid bobruth


### DO THESE FOR THIS SECTION
- fill in the blanks
- add a new leaderboard, and edit the bobbert leaderboard; add proof with a screenshot of the sqlite.db
- add a new set of keys and values to the leaderboard; add proof with a screenshot of the sqlite.db

## JWT (Nikhil)

## Picture Database (Ethan T.)
- ![image](https://user-images.githubusercontent.com/111910633/233181629-36a561c0-8ba5-4644-ac43-274c74a55dff.png)
- Used CRUD methods which have create, read, update, and delete rows in the table.  There is a function called initEasyImages which populates the 'Images' table with data.
- The __init__ method is used to create a new row in the table with a given imagePath, xCoord, yCoord, and difficulty. The __repr__ method returns a string representation of the object when it is printed.
-  The table has five columns: id, _imagePath, _xCoord, _yCoord, and _difficulty.
![image](https://user-images.githubusercontent.com/111910633/233180602-c81c1931-85a4-4d8e-b738-a091eb803d60.png)
- ![image](https://user-images.githubusercontent.com/111910633/233181071-4442d4c3-8c54-497f-b3bf-d625e6742c05.png)
- This part of the code defines getter and setter methods for the columns in the Images model.  It sets and retrieves the metadata of the image.
- ![image](https://user-images.githubusercontent.com/111910633/233181852-9daeed5a-6ab7-4d46-8069-8bcf390ea07a.png)
- The initEasyImages function initializes the database with image metadata for easy difficulty images.


### Picture Encoding (Alex)
The image encoding process for this project is done within the endpoints created on our backend application. The overarching process is split into three seperate sub-processes, each handling a different level of game-difficulty. For this lesson, we will look at the easy images sub-process as our example.

```python
def get_random_easy_image():
    images = Images.query.filter_by(_difficulty=0).all()
    image = random.choice(images)
    return image
```

This procedure is a void function that takes in no arguments directly, but does access the database via the `Images` object imported from the models that we created  

The first line filters all database entries and searches for all image entries with the difficulty column set to 0, which we established as the identifying factor for easy images. The `.all()` method attached to the end of the query serves to return all matching image objects in a list called `image`.

The procedure finally returns a random image using the `random.choice` method.

```python
class ImagesAPI:
    class _EasyImages(Resource):
        def get(self):
            image = get_random_easy_image()
            json_data = {}
            if image:
                image_path = project_path + "/" + image.imagePath
                with open(image_path, "rb") as image_file:
                    json_data["bytes"] = str(base64.b64encode(image_file.read()))[2:][:-1]
                json_data["xCoord"] = image.xCoord
                json_data["yCoord"] = image.yCoord
            return jsonify(json_data)
```

If the first procedure locates and selects the image, the second procedure then encodes, formats, and returns the JSON data containing the image and its metadata. 

The procedure first calls the `get_random_easy_image()` procedure created earlier to store our image object and also initializes an empty dictionary to store our json data.

If a valid image object is found, the procedure will attempt to create an absolute path to the image file on the computer running flask application. This is achieved by concatenating the absolute path of the working project directory to the relative image path stored in the database. The aboslute path of the directory is generated in our _namespace package_ (\_\_init\_\_.py) in our _nighthawkguessr\_api_ package. The line calculating the project path is shown below, and uses the `Path` object from the `pathlib` library.

```python
project_path = Path.cwd().as_posix()
```

After creating the image path, the procedure attempts to open the image file, and procedes to use the `base64.b64encode` method to encode the image file bytes into base64 data. The base64 bytes are then encoded into a python literal string in order to be jsonified and displayed on the api endpoint. The `[2:][:-1]` appended to the end is a type of string slicing which removes the **b'** generated at the start of the data and the **'** generated at the end of the data. These extraneous characters are used to indicate a base64 string, and are not part of the original data.

The procedure finally appends the metadata (position of the right coordinates on the map) to the `json_data` dictionary, completing the process.

Finally, the procedure returns the JSONified data, which is accessible from our API. As you can see, the images we process are extremely large and possess high resolution (typically a few megabytes!)
!["API endpoint"](./lessonimages/image_api_results.png)

In [ ]:
from sqlalchemy import Column, Integer, String, Text, LargeBinary
from sqlalchemy.exc import IntegrityError
from pathlib import Path

class Images(db.Model):
    __tablename__ = 'images'
    id = Column(Integer, primary_key=True)
    _imagePath = Column(Text, unique=True, nullable=False)
    _xCoord = Column(Integer, nullable=False)
    _yCoord = Column(Integer, nullable=False)
    _difficulty = Column(Integer, nullable=False)

    
    def __init__(self, imagePath, xCoord, yCoord, difficulty):
        self._imagePath = imagePath
        self.xCoord = xCoord
        self.yCoord = yCoord
        self.difficulty = difficulty

    def __repr__(self):
        return "<image(id='%s', imagePath='%s', xCoord='%s', yCoord='%s', difficulty='%s')>" % (
            self.id,
            self.imagePath,
            self.xCoord,
            self.yCoord,
            self.difficulty
        )
    @property
    def imagePath(self):
        return self._imagePath

    @imagePath.setter
    def imagePath(self, value):
        self._imagePath = value

    @property
    def xCoord(self):
        return self._xCoord

    @xCoord.setter
    def xCoord(self, value):
        self._xCoord = value

    @property
    def yCoord(self):
        return self._yCoord

    @yCoord.setter
    def yCoord(self, value):
        self._yCoord = value

    @property
    def difficulty(self):
        return self._difficulty

    @difficulty.setter
    def difficulty(self, value):
        self._difficulty = value

    def to_dict(self):
        return {"id": self.id, "imagePath": self._imagePath, "xCoord": self._xCoord, "yCoord": self._yCoord, "difficulty": self._difficulty}

    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "path": self.imagePath,
            "xCoord": self.xCoord,
            "yCoord": self.yCoord,
            "difficulty": self.difficulty
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, path="", xCoord="", yCoord="", difficulty=""):
        """only updates values with length"""
        xCoord = int(xCoord)
        yCoord = int(yCoord)
        if path:
            self.imagePath = path
        if xCoord >= 0:
            self.xCoord = xCoord
        if yCoord >= 0:
            self.yCoord = yCoord
        if difficulty in range(3):
            self.difficulty = difficulty
        db.session.commit()
        return self


    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

In [ ]:
def initEasyImages():
    with app.app_context():
        db.create_all()
        image_dir = Path.cwd()/"images/easy"
        images_paths = [i.name for i in image_dir.iterdir()]
        images = [Images("images/easy/" + image, 250, 250, 0) for image in images_paths]
        for image in images:
             try:
                image.create()
                print("Successfully added entry")
             except:
                 db.session.remove()
                 print("Error adding image: ", image.imagePath)

initEasyImages()

Successfully added entry


## Connection of Databases from Frontend to Backend (David)
We will establish endpoints in the backend, which will ultimately connect to the frontend

The backened takes note of the SQLite database that we use, and runs certain queries over it to filter and return valid values. Additionally, we also used a dictionary data structure to represent the key and value pairs in our JSON data, and also used a list to represent all the valid database entires that we can select from. By creating an API endpoint on our backend, we can thus retrieve this jsondata in the frontend, and display the necessary image or update the necessary variables that is required for program execution

### Methods in Backend

In [ ]:
from flask import Blueprint, request
from flask_restful import Api, Resource, reqparse

leaderboard_bp = Blueprint("leaderboards", __name__)
leaderboard_api = Api(leaderboard_bp)

def get_user_list():
    users_list = [[user._username, int(user._pointsEasy)+2*int(user._pointsMedium)+3*int(user._pointsHard)] for user in Leaderboard.query.all()]
    return users_list

def find_by_username(username):
    users = Leaderboard.query.filter_by(_username=username).all()
    return users[0]


class LeaderboardAPI(Resource):
    def get(self):
        username = request.get_json().get("username")
        print(username, "uid")
        user = find_by_username(username)
        if user:
            return user.to_dict()
        return {"message": user}, 404

    def post(self):
        parser = reqparse.RequestParser()
        parser.add_argument("username", required=True, type=str)
        parser.add_argument("password", required=True, type=str)
        parser.add_argument("pointsEasy", required=True, type=int)
        parser.add_argument("pointsMedium", required=True, type=int)
        parser.add_argument("pointsHard", required=True, type=int)
        args = parser.parse_args()

        leaderboard = Leaderboard(args["username"], args["password"],
                                  args["pointsEasy"], args["pointsMedium"], args["pointsHard"])
        try:
            db.session.add(leaderboard)
            db.session.commit()
            return leaderboard.to_dict(), 201
        except Exception as e:
            db.session.rollback()
            return {"message": f"server error: {e}"}, 500

    def put(self):
        username = request.get_json().get("username")
        print(username, "uid")

        try:
            user = find_by_username(username)
            if user:
                user.pointsEasy = int(request.get_json().get("pointsEasy"))
                user.pointsMedium = int(request.get_json().get("pointsMedium"))
                user.pointsHard = int(request.get_json().get("pointsHard"))
                db.session.commit()
                return user.to_dict(), 201
            else:
                return {"message": "leaderboard not found"}, 404
        except Exception as e:
            db.session.rollback()
            return {"message": f"server error: {e}"}, 500

    def delete(self):
        username = request.get_json().get("username")
        print(username, "uid")

        try:
            user = find_by_username(username)
            if user:
                db.session.delete(user)
                db.session.commit()
                return user.to_dict()
            else:
                return {"message": "leaderboard not found"}, 404
        except Exception as e:
            db.session.rollback()
            return {"message": f"server error: {e}"}, 500


class LeaderboardListAPI(Resource):
    def get(self):
        try:
            leaderboards = db.session.query(Leaderboard).all()
            return [leaderboard.to_dict() for leaderboard in leaderboards]
        except Exception as e:
            db.session.rollback()
            return {"message": f"server error: {e}"}, 500

    def delete(self):
        try:
            db.session.query(Leaderboard).delete()
            db.session.commit()
            return []
        except Exception as e:
            db.session.rollback()
            return {"message": f"server error: {e}"}, 500

### Frontend and How it Communicates

In [ ]:
const isLocalhost = Boolean(
        window.location.hostname === "localhost" ||
        window.location.hostname === "[::1]" ||
        window.location.hostname.match(/^127(?:\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}$/)
      )
      const api = isLocalhost ? "http://localhost:8200" : "";

      function addData(username, password, pointsEasy, pointsMedium, pointsHard){

      let data = {
        "username": username,
        "password": password,
        "pointsEasy": pointsEasy,
        "pointsMedium": pointsMedium,
        "pointsHard": pointsHard
      }

      fetch(api + '/leaderboard', {
        method: 'POST',
        headers: {
          'Content-Type': 'application/json',
        },
        body: JSON.stringify(data),
      })
        .then((response) => response.json())
        .catch((error) => {
          console.error('Error:', error);
        });
      }

      const getList = async () => {
        const list = await fetch(api + "/leaderboardTop10").then((r) => r.json());
        let users = Object.entries(list);
        console.log(users)
        return list
      };

      getList().then(list => {
        list.forEach(cls => {
          addTask(cls.key, cls.value)
        })
      })

      function addTask(key, value) {
  
        var tableCells = [key, value]
        var row = document.createelement('tr')
        for (var i = 0; i < tableCells.length; i++) {
          var tableCell = document.createelement('th')
          tableCell.textContent = tableCells[i]
          tableCell.className = 'cell'
          if (i === 0) {
            tableCell.id = 'period'
          } else if (i === 1) {
            tableCell.id = 'class'
          } else if (i === 2) {
            tableCell.id = 'classNum'
          } else if (i === 3) {
            tableCell.id = 'classStart'
          } else if (i === 4) {
            tableCell.id = 'classEnd'
          }
          row.appendChild(tableCell)
        }
        schedule.appendChild(row)
      }

      getList()

### Leaderboard

>CRUD is an acronym used to describe the processes needed in a functional program. These include being able to create, read, update, and delete data based on the database used. This allows total control over data sets and allows fro total manipulation of the dataset.

#### Create

![]({{site.baseurl}}/lessonimages/create.png " ")

Parser is used to look through dataset and make sure there are no duplicates and then to make a new data entry.

#### Read

![]({{site.baseurl}}/lessonimages/read.png " ")

Parser is used in order to look through dataset for all values and send them to frontend.

#### Update

![]({{site.baseurl}}/lessonimages/update.png " ")

Parser is used to look for specific data value you are searching for and then update another specific value based on that found data fragment.

#### Delete

![]({{site.baseurl}}/lessonimages/delete.png " ")

Parser is used to look for specific data value you are searching for and then delete that found data fragment.

### How Images are Called and Displayed on Frontend

In [ ]:
<body style = "background-repeat: no-repeat; background-attachment: fixed; background-size: cover;">
</body>
<script>
const isLocalhost = Boolean(
  window.location.hostname === "localhost" ||
  window.location.hostname === "[::1]" ||
  window.location.hostname.match(/^127(?:\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}$/)
)
const api = isLocalhost ? "http://localhost:8200" : "";

const getList = async () => {
  const list = await fetch(api + "/api/images/GetEasyImage").then((r) => r.json());
  return list
};

getList().then(list => {
  document.getElementsByTagName("body")[0].style = "background-image: url('data:image/png;base64, " + list.bytes +"');"
})
</script>

The bytes of the images are called into the frontend with a fetch command, which pulls the data either from the locally or globally run flask server. The data is stored in ```list``` and is then made into a variable that is inserted directly into the HTML style of the body of the document, with the user of a css function that converts base64 into an image.

## Time and Space Complexity of Algorithms (Ethan Z, Alex)
The lesson for time and space complexity of algorithms will consist of demonstrations of sorting algorithms and the different time complexities that they come with on a small scale. This can be acomplished using things such as a deck of cards or even with actual people. An example of how this will work is laying out the cards in a random order on the table, and demonstrating different ways of sorting it. From methods such as bubble sort to methods such as bogo sort, it will be really easy to see exactly how much time it would take for these different sorting algoritms to complete. Then after that, it is easy to understand the concept of time complexity when given a real world example.

Space complexity can be demonstrated by using the same method, however, adding in the extra step of having the cards in a pile. This will show how much space is needed to complete the sorting algorithm. This will easily reveal the concept of space complexity because it allows people to witness a real world, physical example of the concept, turning a really hard to grasp concept into something that is easy to understand.

Additionally, we are also going to analyze certain algorithms within our project to show how such analysis is applicable to real world projects.


### QuickSort Algorithm
The main algorithm we've employed is an recursive implementation of quickSort in the backend to sort the database entries to obtain the user with the highest overall score. Typically this algorithm is implemented to sort in ascending order, but to make it easier to extract the top 10 users, we sill be modifying the algorithm to sort in descending order (so the top players show up first in the list).

```python
class LeaderboardTop10(Resource):
    def partition(self, arr, lo, hi):
        pivot = arr[hi][1]
        pivot_pos = lo - 1
        for idx in range(lo, hi):
            if arr[idx][1] >= pivot:
                pivot_pos+=1
                arr[pivot_pos], arr[idx] = arr[idx], arr[pivot_pos]
        arr[pivot_pos + 1], arr[hi] = arr[hi], arr[pivot_pos + 1]
        return pivot_pos+1
    
    def qSortUserList(self, arr, lo, hi):
        if lo < hi:
            part = self.partition(arr, lo, hi)
            self.qSortUserList(arr, lo, part-1)
            self.qSortUserList(arr, part+1, hi)

    def get(self):
        users_list = get_user_list()
        top10 = {}
        self.qSortUserList(users_list, 0, len(users_list)-1)
        for user in users_list:
            top10[user[0]] = user[1]
        print(top10)
        if len(top10) <= 10:
            return top10
        return top10[:10]
```

To understand how the algorithm first, we must first familiarize ourselves with the process  
```
┌─────────────────────┐               ┌──────────────────┐              ┌─────────────────┐  
│                     │               │                  │              │                 │  
│                     │               │      Split       │              │    display      │  
│      Database       │               │       the        ├──────────────►    sorted list  │  
│                     │               │    List into     │              │    on endpoint  │  
│                     │               │   top 10 users   │              │                 │  
└──────────┬──────────┘               └─────────▲────────┘              └─────────────────┘  
           │                                    │  
           │                                    │  
           │                                    │  
           │                                    │  
┌──────────▼──────────┐               ┌─────────┴────────┐  
│                     │               │                  │  
│   2-D User  Array   │               │    QuickSort     │  
│                     ├───────────────►                  │  
│ [(username, score)] │               │    Algorithm     │  
│                     │               │                  │  
└─────────────────────┘               └──────────────────┘  
```
The quickSort algorithm consists of 2 procedures, a `partition()` procedure that **pivots** and *sorts* (not really) our list, and a `qSortUserList()` that performs recursion and actually finishes the sorting.

### The Logic
The partition procedure will serve to "partition" our list into 2 parts, a smaller portion and a larger portion around a certain pivot value (Note: These portions don't have to sorted yet!). 

#### Partition procedure
First, partition selects a certain element in a list to be a pivot. This pivot value will be used to make comparisons to every other value in the array. For simplicity, we have selected our pivot element as the last element in the unsort list. The procedure also defines a `pivot_pos` variable to denote the correct position of the pivot index after each iteration. Although it may seem weird that the variable intially starts with a value of 0-1 = -1, this negative index is resolved by the return statement, which always returns an index that's one higher than pivot_pos. This serves to ensure that the lowest pivot index would be 0, which is the smallest element in any given partition. 
```python
pivot = arr[hi][1]
pivot_pos = lo - 1
```
The partition will then iterate over each element in the list from the starting and ending indexes indicated by the parameters. if a particular value is found to be larger than the pivot, the algorithm well then increment the recorded pivot position and swap the pivot with the found element. 
```python
for idx in range(lo, hi):
    if arr[idx][1] >= pivot:
        pivot_pos+=1
        arr[pivot_pos], arr[idx] = arr[idx], arr[pivot_pos]
```
Finally, the partition procedure swaps the element at the `pivot_pos+1` index with our pivot element at the `hi` index to place the pivot at it's rightful place. We know that `pivot_pos+1` must be smaller than `hi` because it did not trigger the conditional to increment the `pivot_pos+1`` index. At last, the pivot_pos is incremented and returned to serve as an indicator of where our two partitions split.

```python
arr[pivot_pos + 1], arr[hi] = arr[hi], arr[pivot_pos + 1]
return pivot_pos+1
```

#### qSort procedure
If the parition procedure splits our list into smaller and larger regions, then our qSort procedure ensures that these regions are in order. It's hard to explain, but the procedure iterates over each paritition, repeating the paritition process, setting new pivot values, and correctly places the pivot values at their correct sorted order.

The procedure first checks if the lower index is smaller than the higher index, to ensure that our parition has a length greater than 1. If this check passes, the procedure then calls the `partition` procedure to both split our list into the two distinct regions and also the index of our correctly placed pivot value. The procedure then invokes itself again on the larger interval (replacing hi to be the partition index-1 since the previous pivot is already sorted), and also on the smaller interval (replacing lo to be partition index+1).

Eventually, the qSort procedure will iterate through all elements in the list while parition places each element at the correct position, giving us the final sorted list. 

#### Complexity analysis
To analyze time complexity, we must analyze the partition and qSort procedures.

For the qSort procedure, because we divide the array into 2 halves during each function call, we have a total of log_2(n) function calls, where n is the length of the input array. However, within each fucntion call, we also call the partition procedure, which contains an for-loop that iterates over each element in the partioned array, comparing it to a pivot value. We are well familiar with since loops, and can safely say that runtime scales proportionally to a value m, the length of the partition array. Because the length of each partition is also proportional to the length of the input array, we can also conclude the m is proportional to n, and we can also generalize log_2(n) as simply log(n). Thus, the asymptotic notation for a time complexity could be represented as **O(nlogn)**, where n is the length of the input array. Keep in mind, Big-Oh notation is not a function that returns the accurate runtime for any given input, but rather only gives us a model to see how runtime scales with increasing input sizes.

Space complexity for this program is much easier to analyze. Because the array is sorted in-place (meaning we don't create other placeholders or copies), we require no additional space to sort this array. However, recursive function calls may require extra storage on the call stack. Since we found the number of recursive calls to be asymptotic to O(logn), we can conclude that quickSort has an **O(1)** space complexity for an iterative method, and an **O(logn)** space complexity for a recursive method.

## Hacks

Please clone these two repositories:
- [Frontend](https://github.com/DavidVasilev1/hacks-FE)
- [Backend](https://github.com/DavidVasilev1/hacks-BE)

Directions:

You will be creating a full stack of a text and image database which you will code in the Flask Backend and pull information to sort in the Frontend.

You need to have coded a fully working CRUD in the Flask, which you can show working with Postman, however you only need to show the Frontend reading and displaying data from the backend.

You may use the code we showed to you today in class in order to code the both the Frontend and Backend.

Follow the directions in the comments in the Flask Backend and the ReadMe in the frontend before starting.